In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/llama-2/pytorch/13b/1/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-2/pytorch/13b/1/consolidated.00.pth
/kaggle/input/llama-2/pytorch/13b/1/tokenizer_checklist.chk
/kaggle/input/llama-2/pytorch/13b/1/Responsible-Use-Guide.pdf
/kaggle/input/llama-2/pytorch/13b/1/params.json
/kaggle/input/llama-2/pytorch/13b/1/README.md
/kaggle/input/llama-2/pytorch/13b/1/USE_POLICY.md
/kaggle/input/llama-2/pytorch/13b/1/checklist.chk
/kaggle/input/llama-2/pytorch/13b/1/LICENSE.txt
/kaggle/input/llama-2/pytorch/13b/1/.gitattributes
/kaggle/input/llama-2/pytorch/13b/1/tokenizer.model
/kaggle/input/llama-2/pytorch/13b/1/consolidated.01.pth


Adapted from :

- https://www.kaggle.com/code/lucamassaron/fine-tune-llama-2-for-sentiment-analysis

In [13]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"   #Tells pytorch to use the first GPU
os.environ["TOKENIZERS_PARALLELISM"] = "false" #Do not parallise tokenisation (dont know why)

#suppress warnings
import warnings
warnings.filterwarnings("ignore")


In [14]:
# Using the same versions as the above notebook so nothing breaks
!pip install -q -U accelerate==0.23.0 peft==0.5.0 bitsandbytes==0.41.1 transformers==4.33.1 trl==0.7.2

In [15]:
# All of our imports
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [17]:
model_name = "/kaggle/input/llama-2/pytorch/13b-hf/1"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)

RuntimeError: Failed to import transformers.models.falcon.configuration_falcon because of the following error (look up to see its traceback):
cannot import name 'sanitize_code_revision' from 'transformers.models.auto.configuration_auto' (/opt/conda/lib/python3.10/site-packages/transformers/models/auto/configuration_auto.py)

In [ ]:
model_name = "/kaggle/input/llama-2/pytorch/13b/1"


#Using 16 bit floating point for computation
compute_dtype = getattr(torch, "float16")

#Optimises computation for running in kaggle (not quite sure how it works)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, #Loading in model weights as 4 bit
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype= compute_dtype, #using 16 bit for computation
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", # Automatically distributes processing between gpu and cpu
    quantization_config=bnb_config, 
)
